<a href="https://colab.research.google.com/github/Hitomiblood/ProjectsDataScienceMaster/blob/main/Proyecto_TFM/TestingModelos/YoloV8Medium_100Epochs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Modelo de detección de espermatozoides usando YoloV8 Medium

Se realiza la instalación de las librerías necesarias para el procedimiento:

In [1]:
%%capture
!pip install ultralytics

In [2]:
%%capture
!pip install OpenCV
!pip install Pillow

In [3]:
%%capture
!pip install opencv-contrib-python-headless

Cargamos el DataSet y un video de prueba para realizar la detección:

In [4]:
%%capture
# Carga de DataSet
!wget https://www.dropbox.com/scl/fi/8bjlyvmulkwb4l09lgbzc/dataSet.zip?rlkey=dmqb2t13gv9ywi4g9bsesdq6q&st=7hkm6ksd&dl=0
!wget https://www.dropbox.com/scl/fi/f0wmb3677x28qwt3np0t8/acA1920-155um__22949301__20210628_094625267.avi?rlkey=2lfsr3mspsh5potd64pkb7coc&st=17daydxb&dl=1


In [5]:
%%capture
!mv dataSet.zip?rlkey=dmqb2t13gv9ywi4g9bsesdq6q dataSet.zip
!unzip dataSet.zip

In [6]:
%%capture
!mv acA1920-155um__22949301__20210628_094625267.avi?rlkey=2lfsr3mspsh5potd64pkb7coc test.avi

Cargamos un modelo preentrenado. En este caso usaremos el modelo Medium de Yolo para ver el comportamiento en el proceso de entrenamiento y posteriormente aumentar el tamaño del modelo a entrenar.

In [7]:
from ultralytics import YOLO
import cv2 as cv2
from PIL import Image

# Load a pretrained YOLO model (recommended for training)
model = YOLO('yolov8m.pt')

100%|██████████| 49.7M/49.7M [00:00<00:00, 278MB/s]


In [8]:
import os

# Función para corregir las anotaciones
def fix_annotations(annotation_folder):
    for root, dirs, files in os.walk(annotation_folder):
        for file in files:
            if file.endswith(".txt"):
                file_path = os.path.join(root, file)
                fix_annotation_file(file_path)

# Función para corregir un archivo de anotación específico
def fix_annotation_file(file_path):
    with open(file_path, "r+") as file:
        lines = file.readlines()
        file.seek(0)
        for line in lines:
            parts = line.strip().split()
            if len(parts) > 5:
                parts = parts[:5]  # Mantener solo las primeras 5 columnas
            fixed_line = " ".join(parts) + "\n"
            file.write(fixed_line)
        file.truncate()

# Directorio donde se encuentran los archivos de anotaciones
annotation_folder = "dataSet/labels/val"

# Corregir las anotaciones
fix_annotations(annotation_folder)

# Directorio donde se encuentran los archivos de anotaciones
annotation_folder = "dataSet/labels/test"

# Corregir las anotaciones
fix_annotations(annotation_folder)
# Directorio donde se encuentran los archivos de anotaciones
annotation_folder = "dataSet/labels/train"

# Corregir las anotaciones
fix_annotations(annotation_folder)


In [9]:
# Train the model using the 'coco8.yaml' dataset for 3 epochs
results = model.train(data='dataSet.yaml',
                      epochs=100,
                      imgsz=1200,
                      plots = True,
                      batch=-1)

Ultralytics YOLOv8.2.6 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (NVIDIA L4, 22700MiB)
engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=dataSet.yaml, epochs=100, time=None, patience=100, batch=-1, imgsz=1200, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, 

100%|██████████| 755k/755k [00:00<00:00, 27.6MB/s]


Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1      1392  ultralytics.nn.modules.conv.Conv             [3, 48, 3, 2]                 
  1                  -1  1     41664  ultralytics.nn.modules.conv.Conv             [48, 96, 3, 2]                
  2                  -1  2    111360  ultralytics.nn.modules.block.C2f             [96, 96, 2, True]             
  3                  -1  1    166272  ultralytics.nn.modules.conv.Conv             [96, 192, 3, 2]               
  4                  -1  4    813312  ultralytics.nn.modules.block.C2f             [192, 192, 4, True]           
  5                  -1  1    664320  ultralytics.nn.modules.conv.Conv             [192, 384, 3, 2]              
  6                  -1  4   3248640  ultralytics.nn.modules.block.C2f             [384, 384, 4, True]           
  7                  -1  1   1991808  ultralytics

100%|██████████| 6.23M/6.23M [00:00<00:00, 132MB/s]


AMP: checks passed ✅
WARNING ⚠️ imgsz=[1200] must be multiple of max stride 32, updating to [1216]
AutoBatch: Computing optimal batch size for imgsz=1216
AutoBatch: CUDA:0 (NVIDIA L4) 22.17G total, 0.31G reserved, 0.24G allocated, 21.62G free
      Params      GFLOPs  GPU_mem (GB)  forward (ms) backward (ms)                   input                  output
    25856899       285.4         1.837         31.61         141.1      (1, 3, 1216, 1216)                    list
    25856899       570.9         3.045         33.68         74.34      (2, 3, 1216, 1216)                    list
    25856899        1142         5.662         74.34         112.4      (4, 3, 1216, 1216)                    list
    25856899        2283        10.790         175.1         207.6      (8, 3, 1216, 1216)                    list
    25856899        4567        22.219         392.4         417.3     (16, 3, 1216, 1216)                    list
AutoBatch: Using batch-size 9 for CUDA:0 13.06G/22.17G (59%) ✅


...


100 epochs completed in 2.942 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 52.1MB
Optimizer stripped from runs/detect/train/weights/best.pt, 52.1MB

Validating runs/detect/train/weights/best.pt...
Ultralytics YOLOv8.2.6 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (NVIDIA L4, 22700MiB)
Model summary (fused): 218 layers, 25840339 parameters, 0 gradients, 78.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  15%|█▌        | 2/13 [00:00<00:04,  2.38it/s]

WARNING ⚠️ NMS time limit 2.900s exceeded


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 13/13 [00:29<00:00,  2.25s/it]


                   all        228       8684      0.866      0.885      0.933       0.66
Speed: 0.4ms preprocess, 12.4ms inference, 0.1ms loss, 24.8ms postprocess per image
Results saved to runs/detect/train


In [10]:
metrics = model.val(split = 'test', plots = True)

Ultralytics YOLOv8.2.6 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (NVIDIA L4, 22700MiB)
Model summary (fused): 218 layers, 25840339 parameters, 0 gradients, 78.7 GFLOPs


val: Scanning /content/dataSet/labels/test... 177 images, 0 backgrounds, 0 corrupt: 100%|██████████| 177/177 [00:00<00:00, 1311.59it/s]

val: New cache created: /content/dataSet/labels/test.cache



/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 20/20 [00:10<00:00,  1.90it/s]


                   all        177       2773      0.949      0.961      0.964      0.707
Speed: 0.5ms preprocess, 24.0ms inference, 0.0ms loss, 19.7ms postprocess per image
Results saved to runs/detect/train2


In [11]:
metrics.results_dict

{'metrics/precision(B)': 0.9487342273126161,
 'metrics/recall(B)': 0.9610139445229744,
 'metrics/mAP50(B)': 0.9637303084928325,
 'metrics/mAP50-95(B)': 0.7069896111592905,
 'fitness': 0.7326636808926448}

In [12]:
#model.predict(source = 'dataSet/images/test/image_0.jpg', save = True)
results = model.track(source="test.avi", show=True)

requirements: Ultralytics requirement ['lapx>=0.5.2'] not found, attempting AutoUpdate...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 11.1 MB/s eta 0:00:00

requirements: AutoUpdate success ✅ 5.4s, installed 1 package: ['lapx>=0.5.2']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect

WARNING ⚠️ Environment does not support cv2.imshow() or PIL Image.show()



WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/60) /content/test.avi: 768x1216 5 Mov

In [13]:
%%capture
results = model('test.avi', save=True)



WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/60) /content/test.avi: 768x1216 5 Movils, 13.8ms
video 1/1 (frame 2/60) /content/test.avi: 768x1216 5 Movils, 15.5ms
video 1/1 (frame 3/60) /content/test.avi: 768x1216 5 Movils, 16.5ms
video 1/1 (frame 4/60) /content/test.avi: 768x1216 5 Movils, 16.3ms
video 1/1 (frame 5/60) /content/test.avi: 768x1216 5 Movils, 16.0ms
video 1/1 (frame 6/60) /content/test.avi: 768x1216 5 Movils, 15.8ms
video 1/1 (frame 7/60) /content/test.avi: 768x12

In [17]:
%%capture
results = model('Suppl_1.avi', save=True)



WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/60) /content/Suppl_1.avi: 768x1216 31 Movils, 15.6ms
video 1/1 (frame 2/60) /content/Suppl_1.avi: 768x1216 31 Movils, 13.6ms
video 1/1 (frame 3/60) /content/Suppl_1.avi: 768x1216 31 Movils, 13.3ms
video 1/1 (frame 4/60) /content/Suppl_1.avi: 768x1216 31 Movils, 13.6ms
video 1/1 (frame 5/60) /content/Suppl_1.avi: 768x1216 31 Movils, 15.4ms
video 1/1 (frame 6/60) /content/Suppl_1.avi: 768x1216 31 Movils, 14.0ms
video 1/1 (frame 7/60) /

In [15]:
# Export the model
model.export()

Ultralytics YOLOv8.2.6 🚀 Python-3.10.12 torch-2.2.1+cu121 CPU (Intel Xeon 2.20GHz)

PyTorch: starting from 'runs/detect/train/weights/best.pt' with input shape (1, 3, 1216, 1216) BCHW and output shape(s) (1, 5, 30324) (49.7 MB)

TorchScript: starting export with torch 2.2.1+cu121...
TorchScript: export success ✅ 5.9s, saved as 'runs/detect/train/weights/best.torchscript' (99.4 MB)

Export complete (8.3s)
Results saved to /content/runs/detect/train/weights
Predict:         yolo predict task=detect model=runs/detect/train/weights/best.torchscript imgsz=1216  
Validate:        yolo val task=detect model=runs/detect/train/weights/best.torchscript imgsz=1216 data=dataSet.yaml  
Visualize:       https://netron.app


'runs/detect/train/weights/best.torchscript'

In [18]:
import shutil
archivo = shutil.make_archive('ModelInformation', 'zip','/content/runs')